In [8]:
url = 'http://127.0.0.1:25510/v2/snapshot/stock/trade?root=VOO'
requests.get(url).json()

In [11]:
# us100 thetadata
import requests
import threading
import time
from datetime import datetime, timedelta
import psycopg2
from IPython.display import display, clear_output
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
class ContinuousProcessor:
    def __init__(self):
        self.url = 'http://127.0.0.1:25510/v2/snapshot/stock/trade?root='
        self.conn = self.get_db_connection()
        self.cursor = self.conn.cursor()
        self.load_stock_lists()
        self.market_open = True
    def get_db_connection(self):
        DB_HOST = 'localhost'
        DB_NAME = 'us'
        DB_USER = 'postgres'
        DB_PASS = 'buddyrich134'
        try:
            conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
            return conn
        except Exception as e:
            print(f"Database connection error: {e}")
            raise

    def load_stock_lists(self):
        try:
            # Load US500 stock codes
            self.cursor.execute("SELECT * FROM maincode WHERE listed = 'us500';")
            self.conn.commit()
            self.list_us500 = [i[0] for i in self.cursor.fetchall()]

            # Load US100 stock codes
            self.cursor.execute("SELECT * FROM maincode WHERE listed = 'us100';")
            self.conn.commit()
            self.list_us100 = [i[0] for i in self.cursor.fetchall()]
        except Exception as e:
            print(f"Error loading stock lists: {e}")
            raise

    def fetch_stock_price(self, code, minute):
        try:
            if code == "VOO": print(code)
            url = self.url + code
            res = requests.get(url).json()['response'][0]
            ms = res[0]
            price = res[-2]
            date_str = str(res[-1])
            
            milliseconds_of_day = ms
            seconds_of_day = milliseconds_of_day / 1000
            hours, remainder = divmod(seconds_of_day, 3600)
            minutes, seconds = divmod(remainder, 60)
            minutes = minute
            date = datetime.strptime(date_str, '%Y%m%d')
            result_datetime = date + timedelta(hours=hours, minutes=minutes, seconds=seconds)
            formatted_datetime = result_datetime.strftime('%Y-%m-%d %H:%M:00')
            return [formatted_datetime, code, price]
        except Exception as e:
            return None

    def continuous_loop(self):
        while True:
            try:
                now = datetime.now()
                if self.market_open:
                    list_stockprice = []
                    if now.minute % 5 == 0 and now.second >= 5 and now.second <= 10:
                        for code in self.list_us100[:]:
                            data = self.fetch_stock_price(code, minute=now.minute)
                            if data:
                                list_stockprice.append(data)
                        
                        self.cursor.executemany(
                            "INSERT INTO public.stock_price_5m (da, code, cl) VALUES (%s, %s, %s);",
                            list_stockprice
                        )
                        self.conn.commit()
                        print("Insert success")
                        list_stockprice = []
                        self.calculate()
                        time.sleep(60 - now.second) 
                        clear_output()

                else:
                    time.sleep(5)
                    clear_output()

            except Exception as e:
                pass
    def backtest_code(self, pivoted, code, rolling=5, benchmark_index='QQQ'):
        try:
            pivoted.ffill()
            ret = pivoted.pct_change(fill_method=None)
            ret['stock_price'] = pivoted[code]
            ret['index_price'] = pivoted[benchmark_index]
            ret['ret_diff'] = ret[code] - ret[benchmark_index]
            # type 2
            ret['ret_diff_std'] = ret['ret_diff'].rolling(rolling).std()
            ret['ret_diff_mean'] = ret['ret_diff'].rolling(rolling).mean()
            ret['stdize_ret_diff'] = abs((ret['ret_diff'] - ret['ret_diff_mean'])*ret['ret_diff_std'])*10000

            # # check direction
            ret['lag10'] = ret['stock_price'].shift(5)
            ret['change_stock'] = (ret['stock_price'] - ret['lag10'])/ret['lag10']
            ret['direction'] = ret.apply(lambda x: -1 if x['stock_price'] - x['lag10'] > 0 else 1, axis=1)
            
            # # check if the direction are the same, if not, tned to not being a mean reversion
            ret['lag10_twii'] = ret['index_price'].shift(1)
            ret['change_twii'] = (ret['index_price'] - ret['lag10_twii'])/ret['lag10_twii']
            ret['filter_1'] = ret.apply(lambda x: 1 if abs(x['change_stock'] - x['change_twii']) < abs(x['change_stock']) else 0, axis=1)
            ret['filter_2'] = ret.apply(lambda x: 1 if abs(x['change_stock']) < abs(x['change_twii']) else 1, axis=1)
            last = ret.iloc[-1, :]
            da = last.index[-1]
            if last['stdize_ret_diff'] >= 2 and last['filter_1'] == 1 and last['filter_2'] == 1:
                self.lineNotify('us100_thetadata_5m', code, last['direction'], last['stdize_ret_diff'])
                print(f"{code} direction:, {da} {last['direction']}, f1: {last['filter_1']}, f2: {last['filter_2']}")
    
        except Exception as e:
            print(e, code)
    def lineNotify(self, strat, code, direction, stdized123):
        url = 'https://notify-api.line.me/api/notify'
        line_token = '4YeT18qmmjfGAM40QTmlZqNicCf36sweYK5CAMdEmRS'
        headers = {
            'Authorization': 'Bearer ' + line_token    # 設定權杖
        }
        data = {
            'message':f'{strat}\n{code}\n{direction}\n{datetime.now()}\n{stdized123}'     # 設定要發送的訊息
        }
        data = requests.post(url, headers=headers, data=data)   # 使用 POST 方法
    def calculate(self):
        self.cursor.execute("SELECT da, code, cl from public.stock_price order by da desc;")
        self.conn.commit()
        df = pd.DataFrame(self.cursor.fetchall(), columns=['da', 'code', 'cl'])
        df = df.pivot(values='cl', columns='code', index='da').sort_index(ascending=True)
        index = df.index[-1]
        if len(df) >= 5:
            display(f'Running analysis for index: {self.index_da}')
            for code in self.list_us100:
                if code == "QQQ":
                    continue
                self.backtest_code(df[['QQQ', code]], code)
def main():
    processor = ContinuousProcessor()
    loop_thread = threading.Thread(target=processor.continuous_loop, daemon=True)
    loop_thread.start()

if __name__ == "__main__":
    main()
